In [1]:
import torch as T
import torch.nn as nn
from sklearn.feature_extraction import image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from PIL import Image
from skimage import io
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self, num_classes=2):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.1),
            nn.Conv2d(64, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(p=0.3),
           
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 9 * 9, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1000, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0],-1)
        x = self.classifier(x)
        return x

In [17]:
def bayesian_inference(H, E) :
    # P(A|B) = (P(B|A)*P(A))/P(B)
    # P(A|B) Probabilité à posteriori que l'hypothèse A0 soit vraie (ce que l'on cherche à calculer)
    # P(B|A) Probabilité que l'hypothèse soit vraie, selon le résultat d'un évenement (paramètre E pour évenement)
    # P(A) = Probabilité à prioiri que l'hypothèse A0 est vraie (paramètre H pour hypothèse)
    # P(B) = Probabilité de l'évenement, indépendament de l'hypothèse
         # En supposant que les seuls hypothèse sont A0 et A1 :
         #P(B) = P(B|A0)*P(A0|B) + P(B|A1)
        
    # On ne fait pas très confiance à la preuve 
    confiance = 0.5
    E = E*confiance + (1-confiance)/2
    # biais
    E = E*0.975
    
    
    return (H * E) / (H * E + (1-H)*(1-E))
        
 

In [18]:
# load the trained model
net = T.load("trained_net")
net.to('cpu')
net.eval()

Net(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv2d(64, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.3, inplace=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=41472, out_features=1000, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=1000, out_features=1000, bias=True)
    (5): ReLU(inpl

In [19]:
def predictOneImage(path,img_name,size,number_patches,threshold):
    
    predicted_list = []
    
    # chargement de l'image
    img = io.imread(path+"/"+img_name)
    
    #extraction des patchs
    patches = image.extract_patches_2d(img, (size,size),max_patches=number_patches)
    transform = transforms.Compose([transforms.ToTensor()])
    
    # pour chacun des patch
    for i in range(len(patches)):
        # on applique les mêmes transformations et conditions que pour les images du training
        patches[i] = patches[i].astype('uint16')
        m = np.mean(patches[i])
        if (m>threshold) :
            patch = Image.fromarray(patches[i])
            patch = patch.convert(mode='RGB')
            image_tensor = transform(patch)
            image_tensor = image_tensor.unsqueeze_(0)
            
            # on donne le patch à notre trained model
            output = net(image_tensor)
            
            # on récupère la probabilité que le patch soit issu d'une image de mitochondrie segmentée, 
            #selon l'algorithme
            output = output[0].tolist()
            confidence = F.softmax(T.tensor([[output[0],output[1]]],dtype=T.float),dim=1).tolist()[0][0]
            predicted_list.append(confidence)
                    
        return predicted_list

In [20]:
def predictOne_consistent(path,img_name,size,number_patches,threshold,repeats):
    
    # vu que les patchs sont piochés de manière aléatoire à chaque fois, 
    # j'ai trouvé qu'on obtenait des résultats plus fiables en faisant une moyenne
    # des prédictions sur plusieurs essais sur la même image
    predicted_list = []
    healthy_patches = 0
    fragmented_patches = 0
    for i in range(repeats):
        predicted_list += predictOneImage(path,img_name,size,number_patches,threshold)
    
    
    if len(predicted_list) == 0 :
        print("No prediction can be made")
        return -1,0,0
    consensus_prediction = 0.5
    for prediction in predicted_list :
        if prediction < 0.5 :
            healthy_patches += 1
        else :
            fragmented_patches += 1
        consensus_prediction = bayesian_inference(consensus_prediction, prediction)
        
        
    if consensus_prediction < 0.5:
        print("prediction : HEALTHY (%s)"%(1-consensus_prediction))
    else :
        print("prediction : FRAGMENTED (%s)" %(consensus_prediction))
    return consensus_prediction, healthy_patches, fragmented_patches
    
    

In [38]:
def predictMultiples(path,size,number_patches,threshold,repeats):
    csv = ""
    healthy = 0
    fragmented = 0
    errors = 0
    healthy_patches = 0
    fragmented_patches = 0
    bayesian_prediction = 0.5
    average_prediction = 0
    # pour chaque image du répertoire
    for img_name in listdir(path) :
        res = predictOne_consistent(path,img_name,size,number_patches,threshold,repeats)
        healthy_patches += res[1]
        fragmented_patches += res[2]
        
        """
        # création d'un histogramme
        x = np.array(["Healthy", "Fragmented"])
        if(res == -1) :
            y = np.array([0,0])
        else :
            y = np.array([1-res,res])
        fig, ax = plt.subplots()
        barp = plt.bar(x,y, color=['green', 'red'])
        plt.show()
        # on ouvre et affiche l'image
        im = Image.open(path+"/"+img_name)
        im = im.resize((488,360), Image.NEAREST)
        display(im)
        """
    
        csv += img_name + ";" #Les noms des images contiennent des ",", ne pas en utiliser comme séparateur
        if res[0] == -1 :
            csv += "none;none"
            errors += 1
        else :
            average_prediction += res [0]
            # Inférence Bayésienne :
            bayesian_prediction = bayesian_inference(bayesian_prediction, res[0])  
            if res[0] < 0.5 :
                csv += str(res[0]) + ";healthy"
                healthy += 1
            else :
                csv += str(res[0]) + ";fragmented"
                fragmented += 1
        csv += ";"+ str(res[1])+ ";" + str(res[2])+ "\n" 
        print("============================================================")

    average_prediction /= healthy + fragmented
    head = "image;P(fragmented);Prediction;Healthy patches;Fragmented patches \n"
    
    head += "inférence bayésienne;%s;" %(bayesian_prediction)
    if(bayesian_prediction > 0.5) :
        head += "fragmented;"
    else :
        head += "healthy;"
    head += str(healthy_patches) + ";" + str(fragmented_patches) + "\n"
    
    head += "moyenne;%s;" %(average_prediction)    
    if(average_prediction > 0.5) :
        head += "fragmented;"
    else :
        head += "healthy;"
    head += str(healthy_patches) + ";" + str(fragmented_patches) + "\n"
    
    csv = head + csv
    print(csv)
    
    print(f"il y a eu une prédiction de {healthy} healthy et {fragmented} fragmented sur {healthy + fragmented + errors} images traitées")
    print("confiance : %s" %(bayesian_prediction))
    
    if(errors >0) :
        print("Il y a %s images intraitables" %(errors))
        
        
    fic=open("experimental_result.csv","w")
    fic.write(csv)
    fic.close()

In [41]:
path = "./Data/Images-Cytation/Experiment2_210326/Bad/Prediction"
#path = "./Data/All_bad"
size = 320
number_patches = 10
threshold = 3
repeats = 10

predictMultiples(path,size,number_patches,threshold,repeats)

No prediction can be made
prediction : FRAGMENTED (0.8867505324354478)
prediction : FRAGMENTED (0.883551102929861)
prediction : HEALTHY (0.583200472652361)
prediction : FRAGMENTED (0.8150021257899118)
prediction : FRAGMENTED (0.5546758912503719)
prediction : HEALTHY (0.6492203395557383)
image;P(fragmented);Prediction;Healthy patches;Fragmented patches 
inférence bayésienne;0.8239570854578071;fragmented;9;21
moyenne;0.6512598066995822;fragmented;9;21
E4--1-1-1-Tsf[GFP 469,525]-001.tif;none;none;0;0
D3--1-1-1-Tsf[GFP 469,525]-001.tif;0.8867505324354478;fragmented;2;5
D2--1-1-1-Tsf[GFP 469,525]-001.tif;0.883551102929861;fragmented;1;5
D4--1-1-1-Tsf[GFP 469,525]-001.tif;0.4167995273476391;healthy;3;3
D1--1-1-1-Tsf[GFP 469,525]-001.tif;0.8150021257899118;fragmented;0;5
E2--1-1-1-Tsf[GFP 469,525]-001.tif;0.5546758912503719;fragmented;0;1
E3--1-1-1-Tsf[GFP 469,525]-001.tif;0.35077966044426173;healthy;3;2

il y a eu une prédiction de 2 healthy et 4 fragmented sur 7 images traitées
confiance : 